# Исследование надежности заемщиков


## Оглавление
* [Общая информация о данных](#general)
* [Предобработка данных](#preprocessing)
    * [Удаление пропусков](#drop_na)
    * [Обработка аномальных значений](#anomal)
    * [Удаление пропусков (продолжение)](#drop_na_continue)
    * [Изменение типов данных](#dtype)
    * [Обработка дубликатов](#duplicates)
    * [Категоризация данных](#category)
* [Исследовательский анализ данных](#research)
    * [Зависимость между количеством детей и возвратом кредита в срок](#children)
    * [Зависимость между семейным положением и возвратом кредита в срок](#family)
    * [Зависимость между уровнем дохода и возвратом кредита в срок](#income)
    * [Влияние цели взятия кредита на его возврат в срок](#purpose)
* [Общий вывод](#conclusion)

## Общая информация о данных <a class="anchor" id="general"></a>

**Импортируем библиотеку pandas. Считываем данные из csv-файла в датафрейм и сохраняем в переменную `data`. Путь к файлу:**

`data.csv`

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных <a class="anchor" id="preprocessing"></a>

### Удаление пропусков <a class="anchor" id="drop_na"></a>

**Выведем количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений <a class="anchor" id="anomal"></a>

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение) <a class="anchor" id="drop_na_continue"></a>

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных <a class="anchor" id="dtype"></a>

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов <a class="anchor" id="duplicates"></a>

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных <a class="anchor" id="category"></a>

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные и ответим на вопросы <a class="anchor" id="research"></a>

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок? <a class="anchor" id="children"></a>

Проанализируем суммарное количество должников в зависимости от количества детей у человека.

In [22]:
data.groupby('children')['debt'].agg(sum)

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

Видна четкая отрицательная корреляция между количеством детей и количеством должников. Но могла сложиться такая ситуация, что общее количество кредитов, взятых каждой категорией клиентов, различно. 

Проанализируем % долгов от суммарного количества кредитов, взятых конкретной категорией клиентов.

Напишем функцию, позволяющую на основе абсолютных значений количества должников в каждой категории находить их долю в % от суммарного количества кредитов, взятых конкретной категорией клиентов.

In [23]:
def debt_share_per_category(series, feature):
    for index in series.index:
        series[index] = series[index] / len(data[data[feature] == index]) * 100
        series[index] = f'{series[index]:.{3}f}'
    return series.sort_values(ascending=False)

Применим функцию `debt_share_per_category`, чтобы найти долю должников в % от суммарного количества кредитов, взятых конкретной категорией клиентов по числу детей у них.

In [24]:
debt_share_per_category(data.groupby('children')['debt'].agg(sum), 'children')

children
4    9.756
2    9.454
1    9.235
3    8.182
0    7.544
5    0.000
Name: debt, dtype: object

Видим, что наиболее ненадежными являются клиенты, имеющие 4 детей (9.756% не возвращают деньги в срок). Также складывается впечатление, что клиенты с 5 детьми являются самыми надежными. Но таких людей может быть банально меньшинство, поэтому проанализируем размеры сравниваемых групп.

In [25]:
data['children'].value_counts()

children
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: count, dtype: int64

Можно заметить, что клиентов с 3 - 5 детьми меньше 5% от общего числа заемщиков. По этой причине не стоит говорить о каких-то зависимостях в этих категориях.

**Вывод:** Видно, что наиболее надежными являются клиенты, у которых нет детей (7.000% невозвратов в срок). Самыми ненадежными оказались клиенты с 2 детьми (9.454% невозвратов в срок).

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок? <a class="anchor" id="family"></a>

Проанализируем количество должников в зависимости от семейного положения.

In [26]:
data.groupby('family_status')['debt'].agg(sum)

family_status
Не женат / не замужем    273
в разводе                 84
вдовец / вдова            63
гражданский брак         385
женат / замужем          927
Name: debt, dtype: int64

Больше половины должников приходится на тех, кто зарегистрировал свои отношения. Еще около четверти - люди, находящиеся в гражданском браке. Но может быть и так, что эти категории клиентов просто берут больше кредитов, чем другие. Нужно проанализировать % долгов от общего числа кредитов, взятых конкретной категорией людей.

In [27]:
debt_share_per_category(data.groupby('family_status')['debt'].agg(sum), 'family_status')

family_status
Не женат / не замужем    9.764
гражданский брак         9.313
женат / замужем          7.561
в разводе                7.065
вдовец / вдова           6.625
Name: debt, dtype: object

При более тщательном анализе ситуация изменилась. Наиболее ненадежными являются клиенты, состоящие в гражданском браке, и те, кто не женат/не замужем. Наименьший % должников характерен для вдовцов и вдов.

Проанализируем размеры рассматриваемых категорий клиентов.

In [28]:
data['family_status'].value_counts().sort_values(ascending=False)

family_status
женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: count, dtype: int64

**Вывод:** Наиболее ненадежными являются клиенты, состоящие в гражданском браке (9.313% невозвратов), и те, кто не женат/не замужем (9.000%). Наименьший % должников характерен для вдовцов и вдов (6.625% невозвратов).

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a class="anchor" id="income"></a>

Проанализируем количество должников в зависимости от группы по уровню дохода.

In [29]:
data.groupby('total_income_category')['debt'].agg(sum).sort_values(ascending=False)

total_income_category
C    1353
B     354
D      21
A       2
E       2
Name: debt, dtype: int64

Видно, что наибольшее количество долгов приходится на людей с уровнем дохода от 50 до 200 тыс. руб. Но, возможно, они просто чаще других берут кредит. Проанализируем % долгов от общего количества кредитов, взятых людьми с конкретным достатком.  

In [30]:
debt_share_per_category(data.groupby('total_income_category')['debt'].agg(sum), 'total_income_category')

total_income_category
E    9.091
C    8.498
A    8.000
B    7.060
D    6.017
Name: debt, dtype: object

Теперь видим, что наибольшее количество должников приходится на клиентов с самым низким уровнем дохода: ниже 30 тыс. руб. (9.091% невозвратов). Наиболее надежными оказались клиенты с достатком от 30 до 50 тыс. руб. (6.017% невозвратов).

Оценим размеры сравниваемых групп.

In [31]:
data['total_income_category'].value_counts().sort_values(ascending=False)

total_income_category
C    15921
B     5014
D      349
A       25
E       22
Name: count, dtype: int64

Видно, что клиентов с доходом от 30 до 50 тыс. руб. абсолютное меньшинство, ровно как и самых богатых.

**Вывод:** Наименее надежными являются клиенты с доходом от 50 до 200 тыс. руб. (8.498% невозвратов). Наиболее надежны те, кто располагает доходом 200 тыс. руб. - 1 млн руб. (7.060% невозвратов).

#### 3.4 Как разные цели кредита влияют на его возврат в срок? <a class="anchor" id="purpose"></a>

Посмотрим количество долгов в зависимости от цели взятия кредита.

In [32]:
data.groupby('purpose_category')['debt'].agg(sum).sort_values(ascending=False)

purpose_category
операции с недвижимостью    780
операции с автомобилем      400
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64

Видно, что люди, которые берут кредит на недвижимость, чаще всего не возвращают деньги в срок. Но может быть и такое, что кредиты на недвижимость просто чаще берут, поэтому и % должников от общего количества долгов наибольший.

Теперь посмотрим % долгов от общего количества кредитов, взятых для конкретной цели.

In [33]:
debt_share_per_category(data.groupby('purpose_category')['debt'].agg(sum), 'purpose_category')

purpose_category
операции с автомобилем      9.348
получение образования       9.253
проведение свадьбы          7.912
операции с недвижимостью    7.255
Name: debt, dtype: object

Люди, которые берут кредит на операции с автомобилем или образование, чаще всего не возвращают деньги в срок.

Оценим размеры исследуемых категорий.

In [34]:
data['purpose_category'].value_counts().sort_values(ascending=False)

purpose_category
операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: count, dtype: int64

**Вывод:** Люди, которые берут кредит на операции с автомобилем или образование, действительно, чаще всего не возвращают деньги в срок: 9.000% и 9.253% невозвратов, соответственно. 

#### 3.5 Приведем возможные причины появления пропусков в исходных данных. <a class="anchor" id="missing_values_reason"></a>

*Ответ:* 
Пропуски могли возникнуть случайно, так как кредит вряд ли пердоставили бы без информации о трудовом стаже и уровне дохода.

#### 3.6 Объясним, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* На медианное значение практически не влияют выбросы в данных (неожиданно большие или малые величины). При использовании среднего арифметического колоссальное влияние будут оказывать выбросы, завышающие или занижающие результат.

### Шаг 4: общий вывод. <a class="anchor" id="conclusion"></a>

Сопоставляя влияние различных факторов на возврат кредита в срок, выделены следующие тренды:
* Наиболее надежными являются клиенты, у которых нет детей (7.000% невозвратов в срок). Самыми ненадежными оказались клиенты с 2 детьми (9.454% невозвратов в срок).
* Семейное положение также оказывает влияние на своевременную уплату кредита. Наиболее ненадежными являются клиенты, состоящие в гражданском браке (9.313% невозвратов), и те, кто не женат/не замужем (9.000% невозвратов). Наименьший % должников характерен для вдовцов и вдов (6.625% невозвратов).
* Наименее надежными являются клиенты с доходом от 50 до 200 тыс. руб. (8.498% невозвратов). Наиболее надежны те, кто располагает доходом 200 тыс. руб. - 1 млн руб. (7.060% невозвратов).
* Люди, которые берут кредит на операции с автомобилем или образование, чаще всего не возвращают деньги в срок: 9.000% и 9.253% невозвратов, соответственно.